In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z

--2025-12-13 10:07:29--  https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dn720201.ca.archive.org/0/items/stackexchange/stackoverflow.com-Posts.7z [following]
--2025-12-13 10:07:29--  https://dn720201.ca.archive.org/0/items/stackexchange/stackoverflow.com-Posts.7z
Resolving dn720201.ca.archive.org (dn720201.ca.archive.org)... 64.71.129.148
Connecting to dn720201.ca.archive.org (dn720201.ca.archive.org)|64.71.129.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23026928274 (21G) [application/x-7z-compressed]
Saving to: ‘stackoverflow.com-Posts.7z’

stackoverflow.com-P 100%[===================>]  21.45G  42.3MB/s    in 9m 17s  

2025-12-13 10:16:47 (39.4 MB/s) - ‘stackoverflow.com-Posts.7z’ saved [23026928274/23026928274]



In [ ]:
!apt-get install p7zip-full -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z

--2025-09-17 08:03:52--  https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dn720201.ca.archive.org/0/items/stackexchange/stackoverflow.com-Posts.7z [following]
--2025-09-17 08:03:53--  https://dn720201.ca.archive.org/0/items/stackexchange/stackoverflow.com-Posts.7z
Resolving dn720201.ca.archive.org (dn720201.ca.archive.org)... 64.71.129.148
Connecting to dn720201.ca.archive.org (dn720201.ca.archive.org)|64.71.129.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23026928274 (21G) [application/x-7z-compressed]
Saving to: ‘stackoverflow.com-Posts.7z’

stackoverflow.com-P 100%[===================>]  21.45G  41.6MB/s    in 9m 50s  

2025-09-17 08:13:44 (37.2 MB/s) - ‘stackoverflow.com-Posts.7z’ saved [23026928274/23026928274]



In [ ]:
!apt-get install p7zip-full -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import xml.etree.ElementTree as ET
import subprocess
import pandas as pd
import os
from datetime import datetime
from google.colab import drive

def extract_questions(batch_size=50000, auto_save=True):
    """
    Ekstraksi khusus pertanyaan (PostTypeId=1) tahun 2020-2024
    Kolom: Id, CreationDate, Title, Body, Tags
    """

    try:
        drive.mount('/content/drive', force_remount=True)
    except Exception as e:
        print(f"Error saat menghubungkan Google Drive: {e}")
        return 0, 0, 0
n
    output_folder = '/content/drive/MyDrive/StackOverflow_Data/'
    os.makedirs(output_folder, exist_ok=True)

    batch_files = [f for f in os.listdir(output_folder) if f.startswith('batch_') and f.endswith('.csv')]
    if batch_files:
        batch_nums = [int(f.split('_')[1].split('.')[0]) for f in batch_files]
        last_batch = max(batch_nums)
        skip_count = last_batch * batch_size
        print(f"Resume dari batch {last_batch+1} (skip {skip_count} posts)")
    else:
        last_batch = 0
        skip_count = 0
        print("Mulai dari awal")

    # Variabel
    current_batch = last_batch
    posts_data = []
    processed = 0
    saved = 0

    # Check if the 7z file exists
    if not os.path.exists('stackoverflow.com-Posts.7z'):
        print("Error: File stackoverflow.com-Posts.7z tidak ditemukan")
        return current_batch, processed, saved

    try:
        process = subprocess.Popen(
            ['7z', 'x', '-so', 'stackoverflow.com-Posts.7z'],
            stdout=subprocess.PIPE,
            bufsize=8192
        )
    except FileNotFoundError:
        print("Error: Program 7z tidak ditemukan. Pastikan 7-zip terinstall dan tersedia di PATH")
        return current_batch, processed, saved

    context = ET.iterparse(process.stdout, events=('start', 'end'))
    _, root = next(context)

    try:
        for event, elem in context:
            if event == 'end' and elem.tag == 'row':
                if processed < skip_count:
                    processed += 1
                    elem.clear()
                    root.clear()
                    continue

                post_type = elem.get('PostTypeId', '')
                creation_date = elem.get('CreationDate', '')

                if post_type == '1' and creation_date:
                    try:
                        year = int(creation_date[:4])
                        if 2020 <= year <= 2024:
                            post = {
                                'Id': elem.get('Id'),
                                'CreationDate': creation_date,
                                'Title': elem.get('Title'),
                                'Body': elem.get('Body'),
                                'Tags': elem.get('Tags')
                            }
                            posts_data.append(post)
                    except ValueError:
                        pass

                processed += 1
                elem.clear()
                root.clear()

                if auto_save and len(posts_data) >= batch_size:
                    current_batch += 1
                    df = pd.DataFrame(posts_data)
                    df.to_csv(f'{output_folder}batch_{current_batch}.csv', index=False)
                    saved += len(posts_data)
                    posts_data = []
                    print(f"Batch {current_batch} saved ({saved} total saved)")

    except KeyboardInterrupt:
        print("⏸Process interrupted by user")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if posts_data:
            current_batch += 1
            df = pd.DataFrame(posts_data)
            df.to_csv(f'{output_folder}batch_{current_batch}.csv', index=False)
            saved += len(posts_data)
            print(f"Final batch {current_batch} saved ({len(posts_data)} posts)")

        process.terminate()

        print(f"Finished! Total processed: {processed}, Total saved: {saved}")
        return current_batch, processed, saved

if __name__ == "__main__":
    batch_count, total_processed, total_saved = extract_questions(batch_size=50000)

Mounted at /content/drive
🚀 Mulai dari awal
💾 Batch 1 saved (50000 total saved)
💾 Batch 2 saved (100000 total saved)
💾 Batch 3 saved (150000 total saved)
💾 Batch 4 saved (200000 total saved)
💾 Batch 5 saved (250000 total saved)
💾 Batch 6 saved (300000 total saved)
💾 Batch 7 saved (350000 total saved)
💾 Batch 8 saved (400000 total saved)
💾 Batch 9 saved (450000 total saved)
💾 Batch 10 saved (500000 total saved)
💾 Batch 11 saved (550000 total saved)
💾 Batch 12 saved (600000 total saved)
💾 Batch 13 saved (650000 total saved)
💾 Batch 14 saved (700000 total saved)
💾 Batch 15 saved (750000 total saved)
💾 Batch 16 saved (800000 total saved)
💾 Batch 17 saved (850000 total saved)
💾 Batch 18 saved (900000 total saved)
💾 Batch 19 saved (950000 total saved)
💾 Batch 20 saved (1000000 total saved)
💾 Batch 21 saved (1050000 total saved)
💾 Batch 22 saved (1100000 total saved)
💾 Batch 23 saved (1150000 total saved)
💾 Batch 24 saved (1200000 total saved)
💾 Batch 25 saved (1250000 total saved)
💾 Batch 26